# Functional Alignment
This notebook describes an example how to use the methods for functional alignment. BrainIAK includes several options that depend on the user needs. The most basic method is the [Shared Response Model](https://papers.nips.cc/paper/5855-a-reduced-dimension-fmri-shared-response-model) (SRM). The main idea behind this method is to capture what is common across participants performing the same task. Given data that is synchronized in the temporal dimension across a group of subjects, SRM computes a low dimensional *shared* feature subspace common to all subjects. The method also constructs orthogonal weights to project from the shared subspace to each subject voxel space.

The functional alignment module simplifies the import and interchange among these methods. The module includes the following variations of SRM:
* SRM: A probabilistic version of SRM
* DetSRM: A deterministic version of SRM
* RSRM: Robust SRM for better filtering idiosyncratic components and outliers in data
* SSSRM: Semi-Supervised SRM for introducing partial labelled data  
* FastSRM: A faster version that projects the data into  
* @SAM Conectivity-SRM?



In [6]:
%matplotlib inline
import scipy.io
from scipy.stats import stats
from sklearn.metrics import confusion_matrix
from sklearn.svm import NuSVC
import numpy as np
import brainiak.funcalign.srm

# Data

In [12]:
# @SAM: Here we need to describe the data and load it
# I am assumming that we load it in a variable named "data" and it is a list with each subject's data as a matrix
# data[i] is data for ith subject with dimensions V_i x T, with V_i the number of voxels 
# for subject i and T the number of TRs
n_subjects = 0 # Number of subjects
n_trs = 0 # Total TRs

Once data is loaded, we divide the data into two halves for a two fold validation.
We will use one half for training SRM and the other for testing its performance.
Then, we normalize the data each half.

In [13]:
train_data = []
test_data = []
for subject in range(n_subjects):
    # Take the first half of TRs as training
    train_data.append(np.nan_to_num(stats.zscore(data[subject][:, :n_trs//2], axis=1, ddof=1)))
    # Take the second half of TRs as testing
    test_data.append(np.nan_to_num(stats.zscore(data[subject][:, -(n_trs//2):], axis=1, ddof=1)))

# SRM: Training the model

Next, we train the SRM model on the training data. 
Therefore, we need to define the dimension of the desired feature space.
A good methodology to find a good value is to apply cross-validation.

Also, we need to define the number of iterations that the SRM algorithm should run. Ideally, this number should be high enough so the algorithm converges.

In [ ]:
features = 50  # The dimension of the deature space
n_iter = 20  # Number of iterations to perform.

# Create an SRM object
srm = brainiak.funcalign.srm.SRM(n_iter=n_iter, features=features)

# Fit the SRM data
print('Fitting SRM')
srm.fit(train_data)
print('SRM has been fit')
print(f'Share response shape: {srm.s_.shape[0]} Features x {srm.s_.shape[1]} Time-points')

After training SRM, we obtain a shared response $S$ that contains the values of the features for each TR, and a set of weight matrices $W_i$ that can project from the shared subspace to a specific subject voxel space.

Let us check the orthogonal property of the weight matrix $W_i$ for a subject.
We visualize $W_i^TW_i$ that should be the identity $I$ matrix of shape equal to the number of features we selected.

In [ ]:
subject = 0
print(f'Weight matrix shape: {srm.w_[subject].shape[0]} Voxels x {srm.w_[subject].shape[1]} Features')

plt.title(f'Weight matrix test for orthogonality for subject {subject}') 
plt.figure()
plt.imshow(srm.w_[0].T.dot(srm.w_[0]))
plt.tight_layout()

# Testing the performance of SRM

When we trained SRM above, we learned the weight matrices $W_i$ and the shared response $S$ for the training data. The weight matrices further allow us to convert new data to the shared feature space. We call the `transform` function to convert new data for each subject. Then, we normalize for additional analysis.

In [ ]:
shared_test = srm.transform(test_data)

# Zscore the transformed test data
for subject in range(num_subs):
    shared_test[subject] = stats.zscore(shared_test[subject], axis=1, ddof=1)